In [6]:
import os
import re
import string
import time
from itertools import combinations
import random

In [2]:
def n_shingle_set(word_list, shingle_n):
    # input: a list of string for text
    # output: a set of ngram extracted from the text
    shingle_list = ['#'.join(word_list[i:i+shingle_n])
                    for i in range(len(word_list)-shingle_n+1)]
    return set(shingle_list)

In [3]:
def replace_nonword(text):
    # replace both punctuation and non-ASCII characters
    # return filter(lambda x:x not in string.punctuation, text)
    return re.sub(r'[^\x00-\x7F]','',
                  filter(lambda x:x not in string.punctuation, text))

In [8]:
def seed_gen(n_hashes):
    return [random.randint(0, 2**32-1) for _ in range(n_hashes)]

In [23]:
def min_hash(shingle_set, seed_list):
    init_hash = [hash(x) for x in shingle_set]
    out_put_list = range(len(seed_list))
    for i in range(len(seed_list)):
        out_put_list[i] = min([ele^seed_list[i] for ele in init_hash])
        
    return out_put_list

In [20]:
def jaccard_sim(set1, set2):
    intx = len(set1&set2)
    return intx / float(len(set1) + len(set2) - intx)

In [19]:
def file_to_shingle_list(path, shingle_n):
    # input: file directory
    # output: a list of tuples of (file_name, feature_vector)
    file_shingle_dict = {}
    for file_name in os.listdir(path):
        with open(os.path.join(path, file_name), 'r') as f:
            text = replace_nonword(f.read().lower())
            # print text[:20]
            word_list = re.split("[ \n]+", text)
            file_shingle_dict[file_name.replace('.txt','')] = \
                n_shingle_set(word_list, shingle_n)
    return file_shingle_dict

In [74]:
path = 'corpus-20090418'
shingle_n = 8
# start_time = time.time()
file_shingle_dict = file_to_shingle_list(path, shingle_n)

In [75]:
random.seed(0)
hash_number = 50
seed_list = seed_gen(hash_number)

In [76]:
file_minhash_dict = {k:min_hash(v, seed_list) for k,v in file_shingle_dict.items()}

In [77]:
outlist = map(lambda x: (x[0],x[1], "%0.5f" %
                         jaccard_sim(
                             set(file_minhash_dict[x[0]]),
                             set(file_minhash_dict[x[1]])
                         )),
              combinations(file_minhash_dict.keys(), 2))
outlist.sort(key=lambda line: line[2], reverse=True)
# print("--- %s seconds ---" % (time.time() - start_time))

In [78]:
with open('j_sim_local_hash%d.csv' % hash_number, 'a') as fw:
    for line in [','.join(ele) for ele in outlist]:
        fw.write(line+'\n')

In [51]:
def hash_band(hash_list, band_assignment):
    return [set([hash_list[j] for j in band_assignment[i]]) for i in range(len(band_assignment))]

In [55]:
def count_band(list1,list2):
    return sum([list1[i]==list2[i] for i in range(len(list1))])

In [79]:
n_bands = 10
r=hash_number/n_bands
band_assignment = [[i for i in range(hash_number) if i % n_bands == j] for j in range(n_bands)]

In [80]:
file_hashband_dict = {k:hash_band(v, band_assignment) for k,v in file_minhash_dict.items()}

In [81]:
cand_pair = map(lambda x: (x[0],x[1], "%d" %
                         count_band(
                             file_hashband_dict[x[0]],
                             file_hashband_dict[x[1]]
                         )),
              combinations(file_minhash_dict.keys(), 2))
cand_pair.sort(key=lambda line: line[2], reverse=True)

In [82]:
with open('cand_pair_local_band%d.csv' % n_bands, 'a') as fw:
    for line in [','.join(ele) for ele in cand_pair]:
        fw.write(line+'\n')